In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset, replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/detection/baseline/mmdetection/configs/object/trash.py')

root='/opt/ml/detection/dataset/'

epoch = 'best_bbox_mAP_50_epoch_1'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True
cfg.work_dir = '/opt/ml/detection/baseline/mmdetection/work_dirs/swinL_final'

# cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /opt/ml/detection/baseline/mmdetection/work_dirs/swinL_final/best_bbox_mAP_50_epoch_1.pth
The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([11, 1024]) from checkpoint, the shape in current model is torch.Size([81, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([11]) from checkpoint, the shape in current model is torch.Size([81]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([11, 1024]) from checkpoint, the shape in current model is torch.Size([81, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param with shape torch.Size([11]) from checkpoint, the shape in current model is torch.Size([81]).
size mismatch for roi_head.bbox_head.2.fc_cls.weight: copying a param with shape torch.Size([11, 1024]) from checkpoint, the shape in current model is 

In [6]:
output = single_gpu_test(model, data_loader, show_score_thr=0.0001) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 0.4 task/s, elapsed: 11512s, ETA:     0s

In [47]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


,PredictionString,image_id
0,0 0.40404445 811.8698 2.0483637 1020.4124 27.2...,test/0000.jpg
1,0 0.4725972 952.3398 3.2536342 1020.5143 37.43...,test/0001.jpg
2,0 0.3860508 190.0966 2.7600532 411.8033 51.307...,test/0002.jpg
3,,test/0003.jpg
4,0 0.50428444 1.2887532 4.695411 79.01531 41.47...,test/0004.jpg


In [49]:
import numpy as np
import pandas as pd
import cv2
import os
import random
import matplotlib.pyplot as plt

# class 별 bbox 색
colors = [
    (255, 0, 0), 
    (0, 255, 0),
    (0, 0, 255),
    (127, 127, 0),
    (127, 0, 127),
    (0, 127, 127),
    (200, 200, 200),
    (50, 150, 200),
    (200, 155, 50),
    (130, 198, 20),
]
labels = {0: 'General trash', 1: 'Paper', 2: 'Paper pack', 3: 'Metal', 4: 'Glass', 5: 'Plastic', 6: 'Styrofoam', 7: 'Plastic bag', 8: 'Battery', 9: 'Clothing'}

# 이미지 경로
img_dir = '/opt/ml/detection/dataset/'
# output 파일 경로
output_dir = '/opt/ml/detection/baseline/mmdetection/work_dirs/swinL_final/submission_best_bbox_mAP_50_epoch_1.csv'
# output_dir = '/opt/ml/detection/dataset/train.json'

test_result = pd.read_csv(output_dir)

file_names = test_result['image_id'].values.tolist()
bboxes = test_result['PredictionString'].values.tolist()

idx = 3
# idx = 13
file_name = file_names[idx]
print(file_name)
bbox = bboxes[idx].split()

image = cv2.imread(os.path.join(img_dir, file_name)).astype(np.uint8)
for i in range(0, len(bbox), 6):
    label = int(bbox[i])
    x_min = int(bbox[i + 2].split('.')[0])
    y_min = int(bbox[i + 3].split('.')[0])
    x_max = int(bbox[i + 4].split('.')[0])
    y_max = int(bbox[i + 5].split('.')[0])
    
    # bounding box 그리기
    image = cv2.rectangle(image, pt1=(x_min, y_min), pt2=(x_max, y_max), color=colors[label], thickness=5)
    # label text 넣기
    cv2.putText(image, labels[label], (x_min, y_min - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, colors[label], 3)

plt.figure(figsize=(10,10))
plt.imshow(image)

test/0003.jpg


AttributeError: 'float' object has no attribute 'split'